# Import Libs

In [2]:
import os
import io
import sys
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

# Load openAI key

In [3]:
load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

# Initializing the model

In [4]:
openai = OpenAI()
OPENAI_MODEL = "gpt-4o"

# System Prompt

In [13]:
system_message = "You are an assistant that writes and executes all possible unit tests. "
system_message += "Respond with a markdown table format contains 4-columns an ID, Input values, the test purpose and the result after executing (Pass / Fail). "
system_message += "You don't need to provide any other details with the table. "
system_message += "Your job is to read and understand the purpose of the code then provide as much as possible inputs that can make the programming code fails to handle."
system_message += "Add some emojis for good looking."
system_message += "The table edges must come under each other for example: \n"
system_message += """
| ID  | Input Values                                                                                  | Test Purpose                                      | Result  |
|-----|-----------------------------------------------------------------------------------------------|---------------------------------------------------|---------|
|  1  | emp_id=101, name='John Doe', email='john@company.com', department='Engineering', salary=75000 | ✅ Validate normal registration and display       | Pass    |
|  2  | emp_id=104, name='Alice', email='alice@company.com', department='Finance', salary=80000       | ✅ Validate registration of new employee          | Pass    |
"""

# User Prompt

In [14]:
def get_user_prompt(Code):
    user_message = "Take the following code read it line by line, understand what it does, how it does it, check the programming language that is used. "
    user_message += "Then Answer by a table in a markdown format. Your main purpose is to generate as many as possible inputs to check if the code is perfectly handled.\n\n"
    user_message += Code
    
    return user_message

# Prepare the Prompt

In [15]:
def get_prompt(code):
    system = system_message
    user = get_user_prompt(code)

    return [
        {'role': 'system', 'content': system},
        {'role': 'user', 'content': user}
    ]

# Get Model Response

In [16]:
def test_by_gpt(code):
    prompt = get_prompt(code)
    
    stream = openai.chat.completions.create(model=OPENAI_MODEL, messages=prompt, stream=True)
    reply = ''
    
    for chunk in stream:
        reply += chunk.choices[0].delta.content or ''
    
        yield  reply

    yield reply

# Interface

In [17]:
custom_css = """
/* Button styling */
.white-bg {
   background: linear-gradient(135deg, #667eea, #764ba2) !important;
   color: white !important;
   border: none !important;
   border-radius: 12px !important;
   font-weight: 600 !important;
   padding: 15px 30px !important;
   box-shadow: 0 4px 15px rgba(102, 126, 234, 0.3) !important;
}

.white-bg:hover {
   transform: translateY(-2px) !important;
   box-shadow: 0 8px 25px rgba(102, 126, 234, 0.5) !important;
}

/* Row styling */
.row {
   background: rgba(255, 255, 255, 0.15) !important;
   border-radius: 15px !important;
   padding: 20px !important;
   margin: 10px 0 !important;
   backdrop-filter: blur(10px) !important;
   border: 1px solid rgba(255, 255, 255, 0.2) !important;
}

.card {
    border-radius: 15px !important;
    border: 2px solid #667eea !important;
}
"""

In [18]:
with gr.Blocks(css=custom_css) as UI:
    gr.Markdown("# Unit Tests Generator")
    
    with gr.Row():
        with gr.Column():
            gr.Markdown("## Source Code")
            code = gr.Textbox(label='', lines=10, elem_classes=['card'])
    with gr.Row():
        with gr.Column():
            gr.Markdown("## Test Cases")
            response = gr.Markdown("", elem_classes=['card'])  # Changed from gr.Textbox to gr.Markdown
    with gr.Row():
        convert = gr.Button('Test Code', elem_classes=['white-bg'])
        
    convert.click(test_by_gpt, inputs=[code], outputs=[response])
    
UI.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
